# Day 11
## Part 1

In [1]:
import itertools


DIRECTIONS = {(dr, dc) for dr, dc in itertools.product((0, 1, -1), repeat=2)} - {(0, 0)}


def parse_data(s):
    return [list(cs) for cs in s.strip().splitlines()]


def print_data(data):
    for row in data:
        print(''.join(row))
    print()
     
    
def get_coordinate(data, row, col):
    if 0 <= row < len(data) and 0 <= col < len(data[0]):
        return data[row][col]
    else:
        return '.'
        
        
def new_state_of_coordinate(data, row, col):
    c = get_coordinate(data, row, col)
    nbrs = [get_coordinate(data, row + dr, col + dc) 
            for dr, dc in DIRECTIONS]
    if c == '#' and sum(1 for n in nbrs if n == '#') >= 4:
        return 'L'
    elif c == 'L' and all(x != '#' for x in nbrs):
        return '#'
    else:
        return c
    
    
def seating_round(data):
    return [[new_state_of_coordinate(data, row, col) for col, _ in enumerate(line)]
            for row, line in enumerate(data)]

In [2]:
test_data = parse_data('''L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
''')

print_data(test_data)

L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL



In [3]:
def run_until_stable(data, debug=False):
    if debug:
        print_data(data)
    
    n = 0
    while True:
        new_data = seating_round(data)
        if debug:
            print(f'Round {(n := n + 1)}')
            print_data(new_data)
        if new_data == data:
            return new_data
        else:
            data = new_data
        
    
run_until_stable(test_data, debug=True)

L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL

Round 1
#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##

Round 2
#.LL.L#.##
#LLLLLL.L#
L.L.L..L..
#LLL.LL.L#
#.LL.LL.LL
#.LLLL#.##
..L.L.....
#LLLLLLLL#
#.LLLLLL.L
#.#LLLL.##

Round 3
#.##.L#.##
#L###LL.L#
L.#.#..#..
#L##.##.L#
#.##.LL.LL
#.###L#.##
..#.#.....
#L######L#
#.LL###L.L
#.#L###.##

Round 4
#.#L.L#.##
#LLL#LL.L#
L.L.L..#..
#LLL.##.L#
#.LL.LL.LL
#.LL#L#.##
..L.L.....
#L#LLLL#L#
#.LLLLLL.L
#.#L#L#.##

Round 5
#.#L.L#.##
#LLL#LL.L#
L.#.L..#..
#L##.##.L#
#.#L.LL.LL
#.#L#L#.##
..L.L.....
#L#L##L#L#
#.LLLLLL.L
#.#L#L#.##

Round 6
#.#L.L#.##
#LLL#LL.L#
L.#.L..#..
#L##.##.L#
#.#L.LL.LL
#.#L#L#.##
..L.L.....
#L#L##L#L#
#.LLLLLL.L
#.#L#L#.##



[['#', '.', '#', 'L', '.', 'L', '#', '.', '#', '#'],
 ['#', 'L', 'L', 'L', '#', 'L', 'L', '.', 'L', '#'],
 ['L', '.', '#', '.', 'L', '.', '.', '#', '.', '.'],
 ['#', 'L', '#', '#', '.', '#', '#', '.', 'L', '#'],
 ['#', '.', '#', 'L', '.', 'L', 'L', '.', 'L', 'L'],
 ['#', '.', '#', 'L', '#', 'L', '#', '.', '#', '#'],
 ['.', '.', 'L', '.', 'L', '.', '.', '.', '.', '.'],
 ['#', 'L', '#', 'L', '#', '#', 'L', '#', 'L', '#'],
 ['#', '.', 'L', 'L', 'L', 'L', 'L', 'L', '.', 'L'],
 ['#', '.', '#', 'L', '#', 'L', '#', '.', '#', '#']]

In [4]:
def part_1(data):
    return sum(
        1
        for x in itertools.chain.from_iterable(run_until_stable(data))
        if x == '#'
    )

assert part_1(test_data) == 37

In [5]:
data = parse_data(open('input').read())

In [6]:
part_1(data)

2211

In [7]:
%%timeit
part_1(data)

6.25 s ± 63.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


That is very slow. I might have to rethink this.

## Part 2

My rethink involves downloading pypy once homebrew has sorted itself out.

In [21]:
# Use a generator of zero or one values to find
# the next seat (empty or full) in the specified
# direction
def next_visible_seat(data, row, col, dr, dc):
    m = 1
    while (
        0 <= (r := row + m * dr) < len(data) 
        and 0 <= (c := col + m * dc) < len(data[0])
    ):
        if data[r][c] in ('L', '#'):
            yield data[r][c]
            return
        m += 1

        
def visible_seats(data, row, col):
    for dr, dc in DIRECTIONS:
        yield from next_visible_seat(data, row, col, dr, dc)
        
    
def new_state_of_coordinate_2(data, row, col):
    c = get_coordinate(data, row, col)
    nbrs = list(visible_seats(data, row, col))
    if c == '#' and sum(1 for n in nbrs if n == '#') >= 5:
        return 'L'
    elif c == 'L' and all(x != '#' for x in nbrs):
        return '#'
    else:
        return c
    
    
def seating_round_2(data):
    return [[new_state_of_coordinate_2(data, row, col) for col, _ in enumerate(line)]
            for row, line in enumerate(data)]


def run_until_stable_2(data, debug=False):
    if debug:
        print_data(data)
    
    n = 0
    while True:
        new_data = seating_round_2(data)
        if debug:
            print(f'Round {(n := n + 1)}')
            print_data(new_data)
        if new_data == data:
            return new_data
        else:
            data = new_data

In [22]:
run_until_stable_2(test_data, debug=True)

L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL

Round 1
#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##

Round 2
#.LL.LL.L#
#LLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLLL.L
#.LLLLL.L#

Round 3
#.L#.##.L#
#L#####.LL
L.#.#..#..
##L#.##.##
#.##.#L.##
#.#####.#L
..#.#.....
LLL####LL#
#.L#####.L
#.L####.L#

Round 4
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##LL.LL.L#
L.LL.LL.L#
#.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLL#.L
#.L#LL#.L#

Round 5
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.#L.L#
#.L####.LL
..#.#.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#

Round 6
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.LL.L#
#.LLLL#.LL
..#.L.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#

Round 7
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.LL.L#
#.LLLL#.LL
..#.L.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#



[['#', '.', 'L', '#', '.', 'L', '#', '.', 'L', '#'],
 ['#', 'L', 'L', 'L', 'L', 'L', 'L', '.', 'L', 'L'],
 ['L', '.', 'L', '.', 'L', '.', '.', '#', '.', '.'],
 ['#', '#', 'L', '#', '.', '#', 'L', '.', 'L', '#'],
 ['L', '.', 'L', '#', '.', 'L', 'L', '.', 'L', '#'],
 ['#', '.', 'L', 'L', 'L', 'L', '#', '.', 'L', 'L'],
 ['.', '.', '#', '.', 'L', '.', '.', '.', '.', '.'],
 ['L', 'L', 'L', '#', '#', '#', 'L', 'L', 'L', '#'],
 ['#', '.', 'L', 'L', 'L', 'L', 'L', '#', '.', 'L'],
 ['#', '.', 'L', '#', 'L', 'L', '#', '.', 'L', '#']]

In [23]:
def part_2(data):
    return sum(
        1
        for x in itertools.chain.from_iterable(run_until_stable_2(data))
        if x == '#'
    )

assert part_2(test_data) == 26

In [24]:
part_2(data)

1995

In [25]:
%%timeit
part_2(data)

8.12 s ± 570 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Appendix
#### Debugging for posterity

That's wrong. It's frustrating when the test passes but the input fails. Let's have a look at the other examples.

In [12]:
test_data_2 = parse_data('''.......#.
...#.....
.#.......
.........
..#L....#
....#....
.........
#........
...#.....''')

list(visible_seats(test_data_2, 4, 3))

['#', '#', '#', '#', '#', '#', '#', '#']

In [13]:
test_data_3 = parse_data('''.............
.L.L.#.#.#.#.
.............
''')

list(visible_seats(test_data_3, 1, 1))

[]

In [14]:
test_data_4 = parse_data('''.##.##.
#.#.#.#
##...##
...L...
##...##
#.#.#.#
.##.##.''')

list(visible_seats(test_data_4, 3, 3))

[]

They look ok.

What does the final state look like?

In [20]:
print_data(run_until_stable_2(data))

#L#L#L#L.#L#L.#L#L#L#L#.L#L#L#..L.#L#L#.L#L#L.##L#L#.L#L#L#L#L#L#L#.L#L#L#L#L#L#L#.L.#L#L######
LLLLLLLLLLLLL.LLLLLLLLL.#LLLLLL#L.LL.LLLLLLLL.LLLLLL.LLLLL.LLLLLLLL.LLLLLLLL.LLLL.L#LLLLLL#####
#L#L#L#L.#L#L.#L#L#L#L#.LL#L#LLLL#L#L#L#.L#LL.##L#L#.LL#L#L.#L#L#L#.LL#L#L#L.L#L#LLL.#L#L######
LLLLLLLL.LLLLLLLLLLLLLL.#LLLLL#LL.LLLLL.LLLL#.LLLLLL.#LLLL.LLLLLLLL.#LLLLLL#.LLLLL##.LLLLL#####
#L#L#L#L.#L#L.#L#L#L#L#LLL#L#LLL#.L#L#L.#L#LL.##L#L#.L#L#L#L#L#L#L#.LLL#L#LL.L#L#LLL.#L#L######
LLLLLLLL..LLLL.LLLLLLLLL#L#LLL#.L.#LLLLLLLLL#.L#LLLL.LLLLLLLLLLLLLLLL#LLL#L#.LLLLL##.LLLLL#####
...L..#L#.L...........#....LL........#L#....L..L.......#L#L#.L#.....LL.LL...L..L#L.L......#....
#L#L#LLL.L#L#..LL#L#L#LLL#L#L#L#L.LLLLL.L#L##LL##LL#L#LLLL.L#LLL#L#L#L#L#L#L#L#LLLLL.#L#L#.####
LLLLLL#LLLLL#.L#LLLLLLL#LLLLLLLLL.#L#L#.LLLLLLLLLLLL.LLL#L.LLL#LLLL.LLLLLLLLLLLL#L##.LLLLL#####
#L#L#LLL.#LLLLLLL#L#L#L.L#L#L#L#L.LLLLL.#L#L#L#L#L#L.L#LLL.#.LLLL#L.L#L#L#L#.L#LLLLLL#L#L######
LLLLLL#L.LL#..L#LLLLLLLLL.LLLLLLL.#L#.L.

Well that's obviously wrong. Is it checking the ones on the right?

No, it wasn't. In `next_visible_seat` `len(data)` was being used as the rightmost bound rather than `len(data[0])`.

#### Animated!

In [31]:
from IPython.display import clear_output

def run_until_stable_2(data, debug=True):
    if debug:
        print_data(data)
    
    n = 0
    while True:
        new_data = seating_round_2(data)
        if debug:
            clear_output(wait=True)
            print(f'Round {(n := n + 1)}')
            print_data(new_data)
        if new_data == data:
            return new_data
        else:
            data = new_data
            
part_2(data)

Round 85
#L#L#L#L.#L#L.#L#L#L#L#.L#L#L#..L.#L#L#.L#L#L.#L#L#L.#L#L#L#L#L#L#L.#L#L#L#L#L#L#L.#.L#L#L#L#L#
LLLLLLLLLLLLL.LLLLLLLLL.#LLLLLL#L.LL.LLLLLLLL.LLLLL#.LLLLL.LLLLLLL#.LLLLLLLL.LLLL.LLLLLLLLLLLLL
#L#L#L#L.#L#L.#L#L#L#L#.LL#L#LLLL#L#L#L#.L#LL.#L##LL.##L#LL.#L#L#LL.#L#L#L#L.#L#L#L#.L#L#L#L#L#
LLLLLLLL.LLLLLLLLLLLLLL.#LLLLL#LL.LLLLL.LLLL#.LLLLL#.LLLLL.LLLLLLL#.LLLLLLL#.LLLLLLL.LLLLLLLLLL
#L#L#L#L.#L#L.#L#L#L#L#LLL#L#LLL#.L#L#L.#L#LL.#L##LL.##L#L#L#L#L#LL.##L#L#LL.#L#L#L#.L#L#L#L#L#
LLLLLLLL..LLLL.LLLLLLLLL#L#LLL#.L.#LLLLLLLLL#.LLLLL#.LLLLLLLLL#L#L#LLLLLL#L#.LLLLLLL.LLLLLLLLLL
...L..#L#.L...........#....LL........#L#....L..#.......L#LL#.LL.....#L.#L...L..#L#.L......#....
#L#L#LLL.L#L#..LL#L#L#LLL#L#L#L#L.LLLLL.L#L#L#LLLLLL#L#LL#.LL#L#L#LLLLLLLLLLL#LLLLLL.#LL#L.L#L#
LLLLLL#LLLLL#.L#LLLLLLL#LLLLLLLLL.#L#L#.LLLLLLL#L##L.LL#LL.#LLLLLLL.L#L#L#L#LLL#L#L#.LLLLLLLLLL
#L#L#LLL.#LLLLLLL#L#L#L.L#L#L#L#L.LLLLL.#L#L#LLLLLLL.#LLL#.L.L#L#L#.LLLLLLLL.#LLLLLLLL#L#L#L#L#
LLLLLL#L.LL#..L#LLLLLLLLL.LLLLL

1995